<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#从原始数据中提取所用信息" data-toc-modified-id="从原始数据中提取所用信息-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>从原始数据中提取所用信息</a></span></li><li><span><a href="#将已提取数据转化为推荐系统框架格式的数据：user-item-rating-timestamp" data-toc-modified-id="将已提取数据转化为推荐系统框架格式的数据：user-item-rating-timestamp-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>将已提取数据转化为推荐系统框架格式的数据：user item rating timestamp</a></span></li><li><span><a href="#保存歌单和歌曲信息" data-toc-modified-id="保存歌单和歌曲信息-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>保存歌单和歌曲信息</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span><ul class="toc-item"><li><span><a href="#使用协同过滤" data-toc-modified-id="使用协同过滤-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>使用协同过滤</a></span><ul class="toc-item"><li><span><a href="#建立映射字典" data-toc-modified-id="建立映射字典-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>建立映射字典</a></span></li><li><span><a href="#读取文件和训练" data-toc-modified-id="读取文件和训练-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>读取文件和训练</a></span></li><li><span><a href="#寻找最相近的10个user（此处是歌单）" data-toc-modified-id="寻找最相近的10个user（此处是歌单）-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>寻找最相近的 10 个 user（此处是歌单）</a></span></li><li><span><a href="#针对user进行预测" data-toc-modified-id="针对user进行预测-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>针对 user 进行预测</a></span></li></ul></li><li><span><a href="#使用矩阵分解" data-toc-modified-id="使用矩阵分解-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>使用矩阵分解</a></span></li></ul></li><li><span><a href="#模型存储和载入" data-toc-modified-id="模型存储和载入-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型存储和载入</a></span></li><li><span><a href="#算法评估" data-toc-modified-id="算法评估-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>算法评估</a></span></li></ul></div>

# 从原始数据中提取所用信息

In [ ]:
import json
import sys

def parse_song_line(music_file):
    data = json.loads(music_file)
    name = data['result']['name']
    tags = ','.join(data['result']['tags'])
    subscribed_count = data['result']['subcribedCount']
    if subscribed_count<100:
        return False
    playlist_id = data['result']['id']
    song_info = ''
    songs = data['result']['tracks']
    for song in songs:
        try:
            song_info += '\t'+':::'.join([str(song['id']), song['name'], song['artists'][0]['name'], str(song['popularity'])])
        except:
            continue
    return name+'##'+tags+'##'+str(playlist_id)+'##'+str(subscribed_count)+song_info

def parse_file(in_file, out_file):
    with open(out_file, 'w') as f:
        with open(in_file, 'r', encoding = 'utf-8') as g:
            for line in g.readlines():
                result = parse_song_line(line)
                if result:
                    print(result)
                    f.write(result.encode('utf-8').strip() + '\n')
                
parse_file('./playlist.detail.all/playlistdetail.all.json', './163_music_playlist.txt')

# 将已提取数据转化为推荐系统框架格式的数据：user item rating timestamp

In [ ]:
import json
import sys

def is_null(s):
    return len(s.split(','))>2

def parse_song_info(song_info):
    try:
        song_id, name, artist, popularity = song_info.split(':::')
        return ','.join([song_id, '1.0', '1300000'])
    ##评分时，因为原数据仅有是否收藏，因此分数统一写为1.0分
    except:
        return ''

def parse_playlist_line(in_line):
    try:
        contents = in_line.strip().split('\t')
        name, tags, playlist_id, subscribed_count = content[0].split('##')
        songs_info = map(lambda x: playlist_id+','+parse_song_info(x), contents[1:])
        ##这里是以一个歌单为标准格式中的user
        songs_info = filter(is_null, songs_info)
        return '\n'.join(songs_info)
    except:
        return False
    
def parse_file(in_file, out_file):
    with open(out_file, 'w') as out:
        with open(in_file, 'r', encoding = 'utf-8') as f:
            for line in f.readlines():
                result = parse_playlist_line(line)
                if result:
                    out.wirte(result.encode('utf-8').strip()+'\n')

parse_file("./163_music_playlist.txt", "./163_music_suprise_format.txt")

# 保存歌单和歌曲信息

In [ ]:
import pickle
import sys

def parse_playlist_get_info(in_line, playlist_dic, song_dic):
    contents = in_line.strip().split('\t')
    name, tags, playlist_id, subscribed_count = contents[0].split('##')
    playlist_dic[playlist_id] = name
    for song in contents[1]:
        try:
            song_id, song_name, artist, popularity = song.split(':::')
            song_dic[song_id] = song_name+'\t'+artist
        except:
            print('song format error')
            
def parse_file(in_file, out_playlist, out_song):
    playlist_dic = {}
    song_dic = {}
    with open(in_file, 'r') as f:
        for line in f.readlines():
            parse_playlist_get_info(line, playlist_dic, song_dic)
            with open(out_playlist, 'wb') as f:
                pickle.dump(playlist_dic, f)
            with open(out_song, 'wb') as f:
                pickle.dump(song_dic, f)

parse_file('./163_music_playlist.txt', 'playlist.pkl', 'song.pkl')

# 训练模型

## 使用协同过滤

In [2]:
from surprise import Dataset, Reader, KNNBaseline
import pickle
import json
import os

### 建立映射字典

In [3]:
with open('popular_playlist.pkl', 'rb') as f:
    id_name_dic = pickle.load(f, encoding = 'utf-8')
name_id_dic = {values:key for key, values in id_name_dic.items()}

### 读取文件和训练

In [21]:
file_path = 'popular_music_suprise_format.txt'
reader = Reader(line_format = 'user item rating timestamp', sep = ',')
music_data = Dataset.load_from_file(file_path, reader = reader)
trainset = music_data.build_full_trainset()
#sim_options = {'name':'pearson_baseline', 'user_based':False}
algo = KNNBaseline()
algo.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
print(trainset.n_items, trainset.n_ratings, trainset.n_users)

50539 153250 1076


### 寻找最相近的10个user（此处是歌单）

In [31]:
##映射出内部id
current_playlist = list(name_id_dic.keys())[13]
playlist_id = name_id_dic[current_playlist]
playlist_inner_id = algo.trainset.to_inner_uid(playlist_id)

playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)
print(playlist_neighbors)

[1, 2, 3, 6, 7, 12, 13, 14, 15, 17]


In [48]:
print(' 与歌单%s 相似的10个歌单为\n'%(current_playlist))
for name in [id_name_dic[algo.trainset.to_raw_uid(neighbors)] for neighbors in playlist_neighbors]:
    print(name)

 与歌单成长岁月里100味华语男声 相似的10个歌单为

当过千评论的华语翻唱遇上“原唱”【更新】
第四季中国好声音第三期原唱及翻唱
【华语】暖心物语 纯白思念
简单的爱总是那么吸引人
别让时光偷走你上扬的嘴角
你就是我生命中最美丽的比喻句
『华语/回忆』95后陪伴我中学时期的歌曲
对不起，你是一个好人
暗暗作祟| 不甘朋友不敢恋人
有没有一首歌让你泪流满面


### 针对user进行预测

In [52]:
##歌曲id映射
with open('popular_song.pkl', 'rb') as f:
    song_id_name_dic = pickle.load(f, encoding = 'utf-8')
song_name_id_dic = {values: keys for keys, values in song_id_name_dic.items()}

In [61]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = [item for (item, rating) in user_rating]
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'was_impossible': False} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'was_impossible': False} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'was_impossible': False} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'was_impossible': False} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'was_impossible': False} 可以不可以	丁当
user: 4          item: 941        r_ui = 1.00   est = 1.00   {'was_impossible': False} 秋酿	房东的猫
user: 4          item: 616        r_ui = 1.00   est = 1.00   {'was_impossible': False} 退后	周杰伦
user: 4          item: 942        r_ui = 1.00   est = 1.00   {'was_impossible': False} 阴天	莫文蔚
user: 4        

## 使用矩阵分解

In [63]:
from surprise import NMF, Dataset, evaluate, Reader
import pickle

file_path = 'popular_music_suprise_format.txt'
reader = Reader(line_format = 'user item rating timestamp', sep = ',')
music_data = Dataset.load_from_file(file_path, reader = reader)
algo = NMF()
trainset = music_data.build_full_trainset()
algo.fit(trainset)

In [65]:
with open('popular_song.pkl', 'rb') as f:
    song_id_name_dic = pickle.load(f, encoding = 'utf-8')
song_name_id_dic = {value: key for key, value in song_id_name_dic.items()}

In [67]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = [item for (item, rating) in user_rating]
for song in items:
    print(algo.predict(user_inner_id, song, r_ui = 1.0), song_id_name_dic[trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 可以不可以	丁当
user: 4          item: 941      

# 模型存储和载入

In [68]:
import surprise
surprise.dump.dump('./recommedation.model', algo = algo)
algo = surprise.dump.load('./recommedation.model')

In [70]:
algo

(None,
 <surprise.prediction_algorithms.matrix_factorization.NMF at 0x1e4d24914e0>)

# 算法评估